In [1]:
import sys
import numpy as np  # 导入 numpy 库，用于数值计算和数组操作
import matplotlib.pyplot as plt  # 导入 matplotlib.pyplot 库，用于绘图
import cv2 # 导入opencv2库
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体字体

In [2]:
"""
输入图像并输出基本图像信息
"""
def imgInput(title,url):
    try:
        if url is None:
            raise ValueError(f"输入图像错误: {url}")
    except ValueError as e:
        print(e)
    img = url if isinstance(url, np.ndarray) else cv2.imread(url, 1)  # [1 表示彩色图像, 0 表示灰度图像,-1表示透明]
    print(f"图像形状: {img.shape}")
    print(f"图像通道数: {img.ndim}")
    print(f"图像类型: {img.dtype}")
    print(f"图像大小: {img.size}")
    cv2.imshow(title, img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img

In [3]:
"""
输出图像
"""
def imgOutput(url,img):
    try:
        output=cv2.imwrite(url, img)
        if output ==False:
            raise RuntimeError(f"输出图像错误: {url}")
        print("输出:",url)
    except RuntimeError as e:
        print(e)

In [4]:
"""
图像通道分离合并
"""
def channelOperations(title,img):
    b,g,r=img[:,:,0],img[:,:,1],img[:,:,2] #rgb3通道分离蓝绿红
    img_rgb=cv2.merge([r,g,b])
    img_bgr=cv2.merge([b,g,r])
    
    imgInput(f"{title}_b",g)
    imgInput(f"{title}_g",b)
    imgInput(f"{title}_r",r)
    imgInput(f"{title}_rgb",img_rgb)
    imgInput(f"{title}_bgr",img_bgr)

In [5]:
"""
图像数字加减运算运算
"""
def mathOperations(img,adjust):
    # 加操作
    img_add=cv2.add(img,adjust)
    imgInput("paimeng_add",img_add)
    # 减操作
    img_sub=cv2.subtract(img,adjust)
    imgInput("paimeng_sub",img_sub)

In [6]:
"""
图像逻辑运算
"""
def logicalOperations(img,img_mask):
    # 与操作
    img_and=cv2.bitwise_and(img,img_mask)
    imgInput("paimeng_and",img_and)
    # 或操作
    img_or=cv2.bitwise_or(img,img_mask)
    imgInput("paimeng_or",img_or)
    # 非操作
    img_not=cv2.bitwise_not(img)
    imgInput("paimeng_not",img_not)
    # 异或操作
    img_xor=cv2.bitwise_xor(img,img_mask)
    imgInput("paimeng_img_xor",img_xor)

In [7]:
"""
图像颜色空间转换
"""
def imgColor(img):
    imgInput("paimeng_rgb",cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    imgInput("paimeng_gray",cv2.cvtColor(img,cv2.COLOR_BGR2GRAY))
    imgInput("paimeng_hsv",cv2.cvtColor(img,cv2.COLOR_BGR2HSV))
    imgInput("paimeng_ycrcb",cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb))
    imgInput("paimeng_hls",cv2.cvtColor(img,cv2.COLOR_BGR2HLS))
    imgInput("paimeng_xyz",cv2.cvtColor(img,cv2.COLOR_BGR2XYZ))
    imgInput("paimeng_LAB",cv2.cvtColor(img,cv2.COLOR_BGR2LAB))
    imgInput("paimeng_yuv",cv2.cvtColor(img,cv2.COLOR_BGR2YUV))

In [8]:
"""
插值方法列表：

| 插值方法           | 描述                            | 优点                     | 缺点                   | 典型应用场景                   |
|-------------------|---------------------------------|--------------------------|------------------------|---------------------------------|
| INTER_NEAREST      | 最近邻插值                      | 快速                      | 质量较差                | 马赛克效果、简单的图像缩放       |
| INTER_LINEAR       | 线性插值                        | 速度与质量的平衡          | 中等质量                | 一般的图像缩放，默认方法         |
| INTER_CUBIC        | 三次样条插值                    | 高质量                    | 比线性插值慢            | 高质量图像缩放                 |
| INTER_LANCZOS4     | Lanczos插值                    | 最好质量                  | 最慢                    | 高质量放大图像                   |
| INTER_AREA         | 区域插值                        | 适合缩小图像，保留细节     | 不适合放大操作           | 图像下采样                      |

说明：
1. INTER_NEAREST：使用最近邻的像素值，速度最快，适合生成马赛克效果或进行简单的图像缩放。
2. INTER_LINEAR：默认方法，速度和质量的平衡，适合常规的图像缩放操作。
3. INTER_CUBIC：三次样条插值，图像质量比线性插值更好，适用于对质量有更高要求的场景。
4. INTER_LANCZOS4：基于更大像素邻域的插值方法，放大图像时质量最佳，但计算速度慢。
5. INTER_AREA：通过像素区域重采样，特别适合图像缩小时使用，能保留更多的细节。
"""

def imgSample(title, img, rate):
    """
    对输入图像进行采样处理，并生成指定采样率的马赛克效果图像。
    
    参数:
    title: 输出图像文件的标题（用于生成文件名）
    img: 输入图像（numpy数组格式）
    rate: 采样比例（0 < rate <= 1），或目标采样宽度和高度（当rate大于1时）
    
    处理步骤:
    1. 计算采样后的图像宽度和高度。
    2. 使用最近邻插值对图像进行采样。
    3. 将采样后的图像还原到原始尺寸，以产生马赛克效果。
    4. 保存处理后的图像。
    """
    
    # 获取原图的高度和宽度
    original_height, original_width = img.shape[:2]
    
    # 计算采样后的宽度和高度
    if rate > 1:
        sampled_width = rate
        sampled_height = rate
    else:
        sampled_width = int(original_width * rate)
        sampled_height = int(original_height * rate)
    
    # 采样：将图像缩小到计算得到的尺寸
    sampled_img = cv2.resize(img, (sampled_width, sampled_height), interpolation=cv2.INTER_NEAREST)
    
    # 还原：将采样后的图像放大回原始尺寸
    resize_sampled_img = cv2.resize(sampled_img, (original_width, original_height), interpolation=cv2.INTER_NEAREST)
    
    # 显示处理后的图像
    imgInput(f"{title}_{rate}", resize_sampled_img)


In [9]:
def quantize_image(title,img, rate):
    """
    对图像进行颜色量化，将图像的颜色数减少到指定的 rate。
    
    参数:
    img: 输入图像（numpy数组格式）
    rate: 目标颜色数量（应该是2的幂，如2, 4, 8, 16, 32等）
    
    返回:
    量化后的图像（numpy数组格式）
    """
    # 确保图像是RGB格式
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    # 获取图像的高度、宽度和通道数
    height, width, channels = img_rgb.shape
    
    # 计算每个颜色通道的量化级数
    levels = np.linspace(0, 255, rate, dtype=int)
    
    # 将图像数据展平为二维数组，每行表示一个像素的RGB值
    pixels = img_rgb.reshape(-1, 3)
    
    # 对每个颜色通道进行量化
    quantized_pixels = np.zeros_like(pixels)
    for i in range(3):  # 3个颜色通道：R, G, B
        # 获取当前通道的像素值
        channel = pixels[:, i]
        # 找到最接近的量化级别
        quantized_pixels[:, i] = levels[np.searchsorted(levels, channel, side='right') - 1]
    
    # 将量化后的像素数据恢复为图像格式
    quantized_img = quantized_pixels.reshape((height, width, channels)).astype(np.uint8)
    imgInput(f"{title}_{rate}",quantized_img)

In [10]:
imgInputURL = "./img/paimeng.png"
imgOutputURL = "./img/output/paimeng_out.png"
title = "paimeng"
# 输入图像
img=imgInput(title,imgInputURL)




# # 图像分离合并通道
# channelOperations(title,img)

# # 调整图层
# adjust=np.ones(img.shape,np.uint8)*50
# # 图像数学加减运算
# mathOperations(img,adjust)

# # 加载图像蒙版
# img_mask=imgInput("paimeng_circle","./img/circle_paimeng.png")
# # 图像逻辑运算
# logicalOperations(img,img_mask)
# # 图像颜色空间转换
# imgColor(img)

# # 图像采样
# imgSampe(title,img,64)

## 图像量化
# quantize_image(title,img,24)

cv2.waitKey(0)
cv2.destroyAllWindows()
# 输出图像
# imgOutput("./img/circle_paimeng.png",circleBg)




图像形状: (770, 428, 3)
图像通道数: 3
图像类型: uint8
图像大小: 988680
